In [1]:
import numpy as np
import pandas as pd
import datetime

start_time = datetime.datetime.now()
train_identity = pd.read_csv('../input/train_identity.csv', index_col='TransactionID')
#print(train_identity['TransactionID'].size, train_identity['TransactionID'].unique().size)
#print(train_identity.loc[:, train_identity.isnull().any()])
    
#print(train_identity['id_17'].value_counts())
#for col in train_identity.loc[:, train_identity.isnull().any()].columns:
#    print(train_identity[col].value_counts())

Done


In [2]:
train_transaction = pd.read_csv('../input/train_transaction.csv', index_col='TransactionID')
#print(train_transaction['TransactionID'].size, train_transaction['TransactionID'].unique().size)
#print(train_transaction)

Done


In [3]:
train = pd.concat([train_identity, train_transaction], axis=1)
#print(train)

Done


In [4]:
test_identity = pd.read_csv('../input/test_identity.csv', index_col='TransactionID')

Done


In [5]:
test_transaction = pd.read_csv('../input/test_transaction.csv', index_col='TransactionID')

Done


In [6]:
test = pd.concat([test_identity, test_transaction], axis=1)

Done


*chunks*

In [7]:
train_chunk = train#.sample(frac=0.05, random_state=1)
y_chunk = train_chunk['isFraud']
train_chunk.drop(['isFraud'], axis=1, inplace=True)
y_chunk.value_counts()

train = 0

Feature engineering

In [9]:
nan_cols = set()

In [10]:
for col in train_chunk.loc[:, train_chunk.isnull().any()].columns:
    #print(col)
    vc = train_chunk[col].value_counts(dropna=False, normalize=True)
    if vc.loc[np.nan] > 0.9:
        nan_cols.add(col)

#train_chunk['id_12'].value_counts(dropna=False, normalize=True)

In [11]:
len(nan_cols)#, nan_cols

12

In [12]:
for col in test.loc[:, test.isnull().any()].columns:
    #print(col)
    vc = test[col].value_counts(dropna=False, normalize=True)
    if vc.loc[np.nan] > 0.9:
        nan_cols.add(col)

In [13]:
len(nan_cols)#, nan_cols

12

In [14]:
one_value_cols = set()

In [15]:
for col in train_chunk.columns:
    #print(col)
    vc = train_chunk[col].value_counts(normalize=True)
    if vc[vc.index[0]] > 0.9:
        #print(col)
        #print(vc.index[0], vc[vc.index[0]])
        one_value_cols.add(col)

In [16]:
len(one_value_cols)#, one_value_cols

152

In [17]:
for col in test.columns:
    #print(col)
    vc = test[col].value_counts(normalize=True)
    if vc[vc.index[0]] > 0.9:
        #print(col)
        #print(vc.index[0], vc[vc.index[0]])
        one_value_cols.add(col)

In [18]:
len(one_value_cols)#, one_value_cols

156

In [19]:
drop_this = list(nan_cols.union(one_value_cols))
len(drop_this)

166

*concatenate train+test*

In [20]:
data = pd.concat([train_chunk, test], sort=False)
data.drop(drop_this, axis=1, inplace=True)
size = train_chunk.shape[0]
train_chunk = 0
test = 0

In [21]:
def id30(x):
    if pd.isnull(x):
        return x    
    elif 'windows' in x.lower():
        return 'Windows'
    elif 'ios' in x.lower():
        return 'iOS'
    elif 'mac' in x.lower():
        return 'Mac OS'
    elif 'android' in x.lower():
        return 'Android'
    else:
        return x
    
data['id_30'] = data['id_30'].apply(id30)
data['id_30'].value_counts()

Windows    69777
iOS        38502
Mac OS     25634
Android    11783
Linux       2488
func          21
other         19
Name: id_30, dtype: int64

In [22]:
def id31(x):    
    if pd.isnull(x):
        return x
    elif 'chrome' in x.lower():
        return 'Chrome' 
    elif 'safari' in x.lower():
        return 'Safari'
    elif 'firefox' in x.lower():
        return 'Firefox'
    elif 'ie' in x.lower():
        return 'Internet Explorer'
    elif 'edge' in x.lower():
        return 'Microsoft Edge'
    elif 'samsung' in x.lower():
        return 'Samsung Browser'
    elif 'google' in x.lower():
        return 'Google Search Application'
    elif 'opera' in x.lower():
        return 'Opera'
    elif 'android' in x.lower():
        return 'Android Browser'
    elif 'facebook' in x.lower():
        return 'Facebook'    
    else:
        return 'other'

data['id_31'] = data['id_31'].apply(id31)
data['id_31'].value_counts()

Chrome                       155502
Safari                        69866
Internet Explorer             16372
Firefox                       14393
Microsoft Edge                11985
Samsung Browser                4787
Google Search Application      2146
Opera                           843
other                           470
Android Browser                 420
Facebook                        123
Name: id_31, dtype: int64

In [23]:
data['id_33_hor'] = data['id_33'].apply(lambda x: x if pd.isnull(x) else x.split('x')[0])
data['id_33_vert'] = data['id_33'].apply(lambda x: x if pd.isnull(x) else x.split('x')[1])
data.drop(['id_33'], axis=1, inplace=True)

In [24]:
def devinfo(x):
    if pd.isnull(x):
        return x    
    elif 'rv:' in x.lower():
        return 'rv'
    elif 'sm-' in x.lower():
        return 'SM'
    elif 'moto' in x.lower():
        return 'Moto'
    elif ('huawei' in x.lower()) or ('-l' in x.lower()):
        return 'HUAWEI'
    elif 'lg-' in x.lower():
        return 'LG'
    elif 'pixel' in x.lower():
        return 'Pixel'
    elif 'samsung' in x.lower():
        return 'Samsung'
    elif 'windows' in x.lower():
        return 'Windows'
    elif ('ios device' in x.lower()) or ('macos' in x.lower()) or ('trident' in x.lower()):
        return x
    else:
        return 'other'

data['DeviceInfo'] = data['DeviceInfo'].apply(devinfo)
data['DeviceInfo'].value_counts()

Windows        92763
iOS Device     38502
SM             24583
MacOS          23722
other          14132
Trident/7.0    12330
Moto            8155
rv              7197
HUAWEI          7004
LG              4289
Pixel            582
Samsung          464
Name: DeviceInfo, dtype: int64

In [25]:
def p_email(x):
    if pd.isnull(x):
        return x    
    elif 'gmail' in x:
        return 'gmail'
    if 'yahoo' in x:
        return 'yahoo'
    if 'hotmail' in x:
        return 'hotmail'
    if 'outlook' in x:
        return 'outlook'
    if 'anonymous' in x:
        return 'anonymous'
    if 'aol' in x:
        return 'aol'
    if 'comcast' in x:
        return 'comcast'
    if 'icloud' in x:
        return 'icloud'
    if 'live' in x:
        return 'live'
    else:
        return 'other'
    
data['P_emaildomain'] = data['P_emaildomain'].apply(p_email)
data['P_emaildomain'].value_counts()

gmail        436796
yahoo        186568
hotmail       87414
anonymous     71062
other         54523
aol           52337
comcast       14474
icloud        12316
outlook       10797
live           7296
Name: P_emaildomain, dtype: int64

In [26]:
data['R_emaildomain'] = data['R_emaildomain'].apply(p_email)
data['R_emaildomain'].value_counts()

gmail        119081
hotmail       54875
anonymous     39644
yahoo         24912
other         12200
aol            7239
outlook        5864
comcast        3513
live           3013
icloud         2820
Name: R_emaildomain, dtype: int64

**Missing Values**

In [27]:
def missing_values(df):
    for label, col in df.loc[:, df.isnull().any()].iteritems():    
        missing = df[label].isnull()
        vc = col.value_counts(normalize=True)    
        df.loc[missing, label] = np.random.choice(vc.index, size=col.size-col.count(), p=vc.values)    

In [28]:
#print(train['id_01'].value_counts())
#train_copy = train
#missing_values(train_copy)

In [29]:
#test_copy = test
#missing_values(test_copy)

In [30]:
missing_values(data)

*dummies*

In [31]:
data = pd.get_dummies(data)

In [32]:
data.shape

(1097231, 801)

*scaling*

In [33]:
from sklearn.preprocessing import StandardScaler, scale

data = scale(data)

scaler = StandardScaler()
scaled_train = scaler.fit_transform(train_final)

'\ndata = scale(data)\n\nscaler = StandardScaler()\nscaled_train = scaler.fit_transform(train_final)\n'

In [34]:
print(train_final.shape)
print(test_final.shape)
scaled_test = scaler.transform(test_final)

'\nprint(train_final.shape)\nprint(test_final.shape)\nscaled_test = scaler.transform(test_final)\n'

In [35]:
train_final = data[:size]
test_final = data[size:]

In [36]:
#train_final.to_csv('train_final.csv')

In [37]:
#test_final.to_csv('test_final.csv')

In [38]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

**LogisticRegression**

In [39]:
for c in np.power(10.0, np.arange(-5, 6)):
    print('C={}'.format(c))
    print('Score {}\n'.format(cross_val_score(LogisticRegression(C=c, solver='lbfgs', max_iter=300), scaled_train, y_chunk, cv=cv, scoring='roc_auc').mean()))
    
#best_logreg = 'C=0.001, Score 0.792172393105506'

"\nfor c in np.power(10.0, np.arange(-5, 6)):\n    print('C={}'.format(c))\n    print('Score {}\n'.format(cross_val_score(LogisticRegression(C=c, solver='lbfgs', max_iter=300), scaled_train, y_chunk, cv=cv, scoring='roc_auc').mean()))\n    \nbest_logreg = 'C=0.001, Score 0.792172393105506'\n"

**SVC**

In [40]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

best_svc = [0, 0, 0]
for c in np.power(10.0, np.arange(-5, 6)):
    for g in np.power(10.0, np.arange(-5, 6)):
        print('C={}, gamma={}'.format(c, g))
        svc_score = cross_val_score(SVC(C=c, gamma=g), scaled_train, y_chunk, cv=cv, scoring='roc_auc').mean()
        print('Score {}\n'.format(svc_score))
        if svc_score > best_svc[2]:
            best_svc = [c, g, svc_score]
            
print('C={}, gamma={}, Score {}'.format(best_svc[0], best_svc[1], best_svc[2]))
#best_svc = 'C=0.1, gamma=0.001, Score 0.7483778558809864'

"\nbest_svc = [0, 0, 0]\nfor c in np.power(10.0, np.arange(-5, 6)):\n    for g in np.power(10.0, np.arange(-5, 6)):\n        print('C={}, gamma={}'.format(c, g))\n        svc_score = cross_val_score(SVC(C=c, gamma=g), scaled_train, y_chunk, cv=cv, scoring='roc_auc').mean()\n        print('Score {}\n'.format(svc_score))\n        if svc_score > best_svc[2]:\n            best_svc = [c, g, svc_score]\n            \nprint('C={}, gamma={}, Score {}'.format(best_svc[0], best_svc[1], best_svc[2]))\nbest_svc = 'C=0.1, gamma=0.001, Score 0.7483778558809864'\n"

**Random Forest**

In [41]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

rf_score = []
n = [50, 100, 150, 200]
for k in n:
    print('K={}'.format(k))
    rf_score.append(cross_val_score(RandomForestClassifier(n_estimators=k, random_state=1), train_final, y_chunk, cv=cv, scoring='roc_auc').mean())
    print('Score {}\n'.format(rf_score[-1]))
    
estim = n[np.argmax(rf_score)] #0.8456387249804596

"\nrf_score = []\nn = [50, 100, 150, 200]\nfor k in n:\n    print('K={}'.format(k))\n    rf_score.append(cross_val_score(RandomForestClassifier(n_estimators=k, random_state=1), train_final, y_chunk, cv=cv, scoring='roc_auc').mean())\n    print('Score {}\n'.format(rf_score[-1]))\n    \nestim = n[np.argmax(rf_score)] #0.8456387249804596\n"

**Gradient Boosting**

In [42]:

gb_score = []
n = [100, 125, 150, 175, 200]
lrate = [0.1, 0.05, 0.01]
for k in n:
    for l in lrate:
        print('K={}, Learning rate = {}'.format(k, l))
        gb_score.append(cross_val_score(GradientBoostingClassifier(n_estimators=k, learning_rate=l, random_state=1), train_final, y_chunk, cv=cv, scoring='roc_auc').mean())
        print('Score {}\n'.format(gb_score[-1]))
        
#best_gb = 'K=125, Learning rate = 0.1, Score 0.8395876357222953'

"\ngb_score = []\nn = [100, 125, 150, 175, 200]\nlrate = [0.1, 0.05, 0.01]\nfor k in n:\n    for l in lrate:\n        print('K={}, Learning rate = {}'.format(k, l))\n        gb_score.append(cross_val_score(GradientBoostingClassifier(n_estimators=k, learning_rate=l, random_state=1), train_final, y_chunk, cv=cv, scoring='roc_auc').mean())\n        print('Score {}\n'.format(gb_score[-1]))\n        \nbest_gb = 'K=125, Learning rate = 0.1, Score 0.8395876357222953'\n"

In [43]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15,10))
sns.lineplot(n, rf_score)
#sns.lineplot(lrate, gb_score[:3], label='100')
#sns.lineplot(lrate, gb_score[3:6], label='125')
#sns.lineplot(lrate, gb_score[6:9], label='150')
#sns.lineplot(lrate, gb_score[9:12], label='175')
#sns.lineplot(lrate, gb_score[12:], label='200')

NameError: name 'n' is not defined

<Figure size 1080x720 with 0 Axes>

Prediction

In [44]:
submission = pd.read_csv('../input/sample_submission.csv', index_col='TransactionID')

In [45]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

#cls = RandomForestClassifier(n_estimators=estim, random_state=1)
cls = GradientBoostingClassifier(n_estimators=200, random_state=1)
cls.fit(train_final, y_chunk)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='auto',
                           random_state=1, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [46]:
submission['isFraud'] = cls.predict_proba(test_final)[:, 1]
#submission['isFraud']
submission.to_csv('submission.csv')